In [ ]:
import os
import sys
sys.path.append(f"../src")
from pynxtools_em.parsers.hfive_base import HdfFiveBaseParser, NXEM_VOLATILE_NAMED_HDF_PATHS, NXEM_VOLATILE_SUFFIX_HDF_PATHS
print(os.getcwd())
! mkdir -p prod && mkdir -p log && which python
! pip list | grep pynxtools*
import matplotlib
print(matplotlib.get_backend())

## Example how to find all unit tests

In [ ]:
mk_dir = False
test_case_directories = set()
for root, dirnames, files in os.walk(os.getcwd()):
    for filename in files:  # loop through files in the current directory
        file_name = os.path.join(root, filename)
        if file_name.startswith(f"{os.getcwd()}/data/"):
            test_case_name = file_name[0:file_name.rfind("/")].replace(f"{os.getcwd()}/data/", "")
            if not test_case_name.startswith("/home"):
                test_case_directories.add(test_case_name)
        del file_name
for test_case in sorted(test_case_directories):
    print(test_case)
    for subdir in ["prod", "log", "reference"]:
        fpath = f"{os.getcwd()}/{subdir}/{test_case}"
        if not(os.path.exists(fpath) and os.path.isdir(fpath)):
            if mk_dir:
                os.mkdir(fpath)
        del fpath

## Generate list of all tests for code launch.json

In [ ]:
for test_case in sorted(test_case_directories):
    file_list = os.listdir(f"{os.getcwd()}/data/{test_case}")
    # for file_name in file_list:
    #    print(f'''                     //"tests/data/{test_case}/{file_name}",''')
    # print(f'''                     //"tests/prod/{test_case}/output.nxs"]''')    

## Run tests

In [ ]:
run_parser = True
verbose = True
whitelist = []
blacklist = []
n_tests = 0
bash_script_lines = ["#!/bin/sh", ""]
for test_case in sorted(test_case_directories):
    if len(whitelist) > 0:
        if test_case not in whitelist:
            continue
    if len(blacklist) > 0:
        if test_case in blacklist:
            continue

    print(f"- [ ] {test_case}")  # , {len(file_list)}")
    curr_dir = f"{os.getcwd()}/data/{test_case}"
    file_list = os.listdir(curr_dir)
    out = f"{os.getcwd()}/prod/{test_case}/output.nxs"  # data.{parser_type}.{entry}.nxs"
    stdout = f"{os.getcwd()}/log/{test_case}/stdout.{test_case}.txt"
    stderr = f"{os.getcwd()}/log/{test_case}/stderr.{test_case}.txt"
    bash_script_lines.append(f'''echo "{test_case}"''')
    if len(file_list) == 1:
        in_one = f"{curr_dir}/{file_list[0]}"
        if run_parser:
            # ! dataconverter $in_one --reader em --nxdl NXem --output $out 1>$stdout 2>$stderr
            bash_script_lines.append(f"dataconverter {in_one} --reader em --nxdl NXem --output {out} 1>{stdout} 2>{stderr}")
            n_tests += 1
        del in_one
    elif len(file_list) == 2:
        in_one = f"{curr_dir}/{file_list[0]}"
        in_two = f"{curr_dir}/{file_list[1]}"
        if run_parser:
            # ! dataconverter $in_one $in_two --reader em --nxdl NXem --output $out 1>$stdout 2>$stderr
            bash_script_lines.append(f"dataconverter {in_one} {in_two} --reader em --nxdl NXem --output {out} 1>{stdout} 2>{stderr}")
            n_tests += 1
        del in_one, in_two
    elif len(file_list) == 3:
        in_one = f"{curr_dir}/{file_list[0]}"
        in_two = f"{curr_dir}/{file_list[1]}"
        in_three = f"{curr_dir}/{file_list[2]}"
        if run_parser:
            # ! dataconverter $in_one $in_two $in_three --reader em --nxdl NXem --output $out 1>$stdout 2>$stderr
            bash_script_lines.append(f"dataconverter {in_one} {in_two} {in_three} --reader em --nxdl NXem --output {out} 1>{stdout} 2>{stderr}")
            n_tests += 1
        del in_one, in_two, in_three
    del file_list, curr_dir, out, stdout, stderr
print(f"Ran {n_tests} tests")
with open(f"{os.getcwd()}/run_tests.sh", "w") as fp:
    fp.write("\n".join(bash_script_lines))

## Example how to generate a reference artifact from a NeXus file for unit tests

In [ ]:
whitelist = []
blacklist = []
for test_case in sorted(test_case_directories):
    if len(whitelist) > 0:
        if test_case not in whitelist:
            continue
    if len(blacklist) > 0:
        if test_case in blacklist:
            continue           
    print(f"- [ ] {test_case}")
    fpath_in = f"{os.getcwd()}/prod/{test_case}/output.nxs"
    fpath_out = f"{os.getcwd()}/reference/{test_case}/output.nxs.sha256.ref.yaml"
    try:
        hfive_parser = HdfFiveBaseParser(file_path=fpath_in, hashing=True, verbose=False)
        hfive_parser.get_content()
        hfive_parser.store_hashes(
            blacklist_by_key=NXEM_VOLATILE_NAMED_HDF_PATHS,
            blacklist_by_suffix=NXEM_VOLATILE_SUFFIX_HDF_PATHS,
            file_path=fpath_out)
    except FileNotFoundError:
        print(f"- [ ] >>>> ERROR >>>> {test_case}")
    del fpath_in, fpath_out, hfive_parser

***